# Using the MANN Package to train a Computer Vision and Fully Connected Multitask Neural Network

In this notebook, the MANN package will be used to train a multitask network for three tasks, two of which will be computer vision tasks utilizing a convolutional architecture fed into a few fully-connected layers and one of which will be a tabular task using only a fully-connected architecture.  The convolutional tasks will be trained on both the MNIST Digit and MNIST Fashion datasets, and the third will be the Boston Housing Price dataset.

In [1]:
# Load the MinMaxScaler from Scikit Learn, TensorFlow, numpy, and MANN
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import mann

In [2]:
# Load both the MNIST tasks
(digit_x_train, digit_y_train), (digit_x_test, digit_y_test) = tf.keras.datasets.mnist.load_data()
(fashion_x_train, fashion_y_train), (fashion_x_test, fashion_y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Reshape the images so they have channels and divide by 255 so all values are in [0, 1]
digit_x_train = digit_x_train.reshape(digit_x_train.shape + (1,))/255
digit_x_test = digit_x_test.reshape(digit_x_test.shape + (1,))/255
fashion_x_train = fashion_x_train.reshape(fashion_x_train.shape + (1,))/255
fashion_x_test = fashion_x_test.reshape(fashion_x_test.shape + (1,))/255

# Load the Boston housing data and reshape the targets so they have one column
(boston_x_train, boston_y_train), (boston_x_test, boston_y_test) = tf.keras.datasets.boston_housing.load_data()
boston_y_train = boston_y_train.reshape(-1, 1)
boston_y_test = boston_y_test.reshape(-1, 1)

# Scale the Boston housing data to values between 0 and 1, as measured by the training data
boston_x_scaler = MinMaxScaler()
boston_x_train = boston_x_scaler.fit_transform(boston_x_train)
boston_x_test = boston_x_scaler.transform(boston_x_test)

# Scale the target values within the Boston housing dataset as well
boston_y_scaler = MinMaxScaler()
boston_y_train = boston_y_scaler.fit_transform(boston_y_train)
boston_y_test = boston_y_scaler.transform(boston_y_test)

# Reshape the y data to have one column
digit_y_train = digit_y_train.reshape(-1, 1)
digit_y_test = digit_y_test.reshape(-1, 1)
fashion_y_train = fashion_y_train.reshape(-1, 1)
fashion_y_test = fashion_y_test.reshape(-1, 1)
boston_y_train = boston_y_train.reshape(-1, 1)
boston_y_test = boston_y_test.reshape(-1, 1)

# Create a callback to stop training early
callback = tf.keras.callbacks.EarlyStopping(min_delta = 0.01, patience = 3, restore_best_weights = True)

# Create the model

In [3]:
# Create the input layers
digit_input = tf.keras.layers.Input(digit_x_train.shape[1:])
fashion_input = tf.keras.layers.Input(fashion_x_train.shape[1:])
boston_input = tf.keras.layers.Input(boston_x_train.shape[1:])

# Create the convolutional blocks for the image data
image_x = mann.layers.MultiMaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)([digit_input, fashion_input])
image_x = mann.layers.MultiMaskedConv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(image_x)
image_x = mann.layers.MultiMaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(image_x)
image_x = mann.layers.MultiMaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)([digit_input, fashion_input])
image_x = mann.layers.MultiMaskedConv2D(
    filters = 64,
    kernel_size = 3,
    padding = 'same',
    strides = 1,
    activation = 'relu'
)(image_x)
image_x = mann.layers.MultiMaxPool2D(
    pool_size = 2,
    strides = 1,
    padding = 'valid'
)(image_x)
digit_selector = mann.layers.SelectorLayer(0)(image_x)
fashion_selector = mann.layers.SelectorLayer(1)(image_x)
digit_flatten = tf.keras.layers.Flatten()(digit_selector)
fashion_flatten = tf.keras.layers.Flatten()(fashion_selector)
image_x = mann.layers.MultiMaskedDense(256, activation = 'relu')([digit_flatten, fashion_flatten])

digit_x = mann.layers.SelectorLayer(0)(image_x)
fashion_x = mann.layers.SelectorLayer(1)(image_x)
boston_x = mann.layers.MaskedDense(256, activation = 'relu')(boston_input)
boston_x = mann.layers.MaskedDense(256, activation = 'relu')(boston_x)
boston_x = mann.layers.MaskedDense(256, activation = 'relu')(boston_x)

x = mann.layers.MultiMaskedDense(256, activation = 'relu')([digit_x, fashion_x, boston_x])

digit_selector = mann.layers.SelectorLayer(0)(x)
fashion_selector = mann.layers.SelectorLayer(1)(x)
boston_selector = mann.layers.SelectorLayer(2)(x)

digit_output = mann.layers.MaskedDense(10, activation = 'softmax')(digit_selector)
fashion_output = mann.layers.MaskedDense(10, activation = 'softmax')(fashion_selector)
boston_output = mann.layers.MaskedDense(1, activation = 'relu')(boston_selector)

model = tf.keras.models.Model([digit_input, fashion_input, boston_input], [digit_output, fashion_output, boston_output])
model.compile(
    loss = ['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'mae'],
    optimizer = 'adam'
)
model.summary()

Metal device set to: Apple M1


2021-11-09 08:47:36.781429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-09 08:47:36.781519: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
multi_masked_conv2d_2 (MultiMas [(None, 28, 28, 64), 2560        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
multi_masked_conv2d_3 (MultiMas [(None, 28, 28, 64), 147712      multi_masked_conv2d_2[0][0]  

In [4]:
model = mann.utils.mask_model(
    model,
    80,
    method = 'gradients',
    exclusive = True,
    x = [digit_x_train[:boston_x_train.shape[0], :], fashion_x_train[:boston_x_train.shape[0], :], boston_x_train],
    y = [digit_y_train[:boston_x_train.shape[0], :], fashion_y_train[:boston_x_train.shape[0], :], boston_y_train]
)

# Prepare the model for training the first two tasks only
model.compile(
    loss = ['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'mae'],
    optimizer = 'adam',
    loss_weights = [1, 1, 0]
)

In [5]:
# Train the first two tasks
model.fit(
    [digit_x_train, fashion_x_train, np.zeros((digit_x_train.shape[0], boston_x_train.shape[1]))],
    [digit_y_train, fashion_y_train, np.zeros(digit_y_train.shape[0])],
    epochs = 100,
    batch_size = 128,
    callbacks = [callback],
    validation_split = 0.2
)

2021-11-09 08:47:38.814849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-09 08:47:38.815895: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-11-09 08:47:39.234501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 0.9498 - masked_dense_3_loss: 0.3158 - masked_dense_4_loss: 0.6340 - masked_dense_5_loss: 0.0000e+00

2021-11-09 08:48:42.942439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 71s 188ms/step - loss: 0.9498 - masked_dense_3_loss: 0.3158 - masked_dense_4_loss: 0.6340 - masked_dense_5_loss: 0.0000e+00 - val_loss: 0.4758 - val_masked_dense_3_loss: 0.0901 - val_masked_dense_4_loss: 0.3858 - val_masked_dense_5_loss: 0.0000e+00
Epoch 2/100
375/375 [==============================] - 70s 186ms/step - loss: 0.3930 - masked_dense_3_loss: 0.0715 - masked_dense_4_loss: 0.3215 - masked_dense_5_loss: 0.0000e+00 - val_loss: 0.3614 - val_masked_dense_3_loss: 0.0659 - val_masked_dense_4_loss: 0.2956 - val_masked_dense_5_loss: 0.0000e+00
Epoch 3/100
375/375 [==============================] - 69s 184ms/step - loss: 0.2910 - masked_dense_3_loss: 0.0474 - masked_dense_4_loss: 0.2436 - masked_dense_5_loss: 0.0000e+00 - val_loss: 0.3210 - val_masked_dense_3_loss: 0.0579 - val_masked_dense_4_loss: 0.2631 - val_masked_dense_5_loss: 0.0000e+00
Epoch 4/100
375/375 [==============================] - 69s 184ms/step - loss: 0.2289 - masked_dense_

In [6]:
# Prepare the model for training the third task
model.compile(
    loss = ['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'mae'],
    optimizer = 'adam',
    loss_weights = [0, 0, 1]
)
callback = tf.keras.callbacks.EarlyStopping(min_delta = 0.005, patience = 3, restore_best_weights = True)

In [7]:
# Train the third task
model.fit(
    [digit_x_train[:boston_x_train.shape[0]], fashion_x_train[:boston_x_train.shape[0]], boston_x_train],
    [digit_y_train[:boston_x_train.shape[0]], fashion_y_train[:boston_x_train.shape[0]], boston_y_train],
    epochs = 100,
    batch_size = 32,
    callbacks = [callback],
    validation_split = 0.2
)

Epoch 1/100


2021-11-09 08:55:45.603741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 0.3418 - masked_dense_3_loss: 0.0336 - masked_dense_4_loss: 0.1106 - masked_dense_5_loss: 0.3418

2021-11-09 08:55:47.176022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 2s 150ms/step - loss: 0.3418 - masked_dense_3_loss: 0.0336 - masked_dense_4_loss: 0.1106 - masked_dense_5_loss: 0.3418 - val_loss: 0.2817 - val_masked_dense_3_loss: 0.0131 - val_masked_dense_4_loss: 0.1131 - val_masked_dense_5_loss: 0.2817
Epoch 2/100
11/11 [==============================] - 1s 126ms/step - loss: 0.2082 - masked_dense_3_loss: 0.0322 - masked_dense_4_loss: 0.1089 - masked_dense_5_loss: 0.2082 - val_loss: 0.1762 - val_masked_dense_3_loss: 0.0139 - val_masked_dense_4_loss: 0.1124 - val_masked_dense_5_loss: 0.1762
Epoch 3/100
11/11 [==============================] - 1s 124ms/step - loss: 0.1712 - masked_dense_3_loss: 0.0325 - masked_dense_4_loss: 0.1079 - masked_dense_5_loss: 0.1712 - val_loss: 0.1765 - val_masked_dense_3_loss: 0.0142 - val_masked_dense_4_loss: 0.1126 - val_masked_dense_5_loss: 0.1765
Epoch 4/100
11/11 [==============================] - 1s 124ms/step - loss: 0.1442 - masked_dense_3_loss: 0.0326 - masked_dense_4_loss

In [8]:
# Get the predictions for all tasks and report the performance
digit_preds, fashion_preds, dummy = model.predict([digit_x_test, fashion_x_test, np.zeros((digit_x_test.shape[0], boston_x_test.shape[1]))])
dummy1, dummy2, boston_preds = model.predict([digit_x_test[:boston_x_test.shape[0]], fashion_x_test[:boston_x_test.shape[0]], boston_x_test])
digit_preds = digit_preds.argmax(axis = 1)
fashion_preds = fashion_preds.argmax(axis = 1)

print(f'Multitask Digit Accuracy: {(digit_preds.flatten() == digit_y_test.flatten()).sum()/digit_y_test.flatten().shape[0]}')
print(f'Multitask Fashion Accuracy: {(fashion_preds.flatten() == fashion_y_test.flatten()).sum()/fashion_y_test.flatten().shape[0]}')
print(f'Multitask Boston MAE: {np.abs(boston_preds.flatten() - boston_y_test.flatten()).mean()}')

2021-11-09 08:56:19.784304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Multitask Digit Accuracy: 0.9872
Multitask Fashion Accuracy: 0.8677
Multitask Boston MAE: 0.06855997549033113
